In [2]:
import requests
import pandas as pd
import re
import numpy as np
import dataframe_image as dfi
from datetime import datetime, timedelta
import schedule
import os

In [3]:


path = 'https://mpei.ru/Education/timetable/Pages/table.aspx?mode=list&groupoid=16941&start=2024.02.19'

def aspx_to_df(path_lst):
    table = pd.read_html(path_lst, encoding='utf-8')
    df_table = table[2]
    df_table = df_table.drop([0])
    return df_table
# полная обработка ужасного aspx
def generate_schedule(df_table):
    new_lst = []
    lesson = []
    type_lesson = []
    room = []
    lst = df_table[1].tolist()
    for i in lst:
        i = i.replace('(', '')
        i = i.replace('КР', 'Кр')
        if len(re.findall(r'[А-Я][^А-Я]*', i)) < 2:
            lesson.append(0)
            type_lesson.append(0)
            room.append(0)
        else:
            step_1 = re.findall(r'[А-Я][^А-Я]*', i)
            lesson.append(step_1[0])
            type_lesson.append(step_1[1])
            room.append(step_1[2])     
    df_table['lesson'] = pd.Series(lesson)
    df_table['type'] = pd.Series(type_lesson)
    df_table['room'] = pd.Series(room)
    df_table = df_table.drop(1, axis=1)
    df_table[['lesson', 'type', 'room']] = df_table[['lesson', 'type', 'room']].shift(1)
    df_table.replace(0, np.nan, inplace=True)
    df_table = df_table.rename(columns=({0:'Дата и время', 'lesson':'Предмет','type':'Вид занятия', 'room':'Аудитория'}))
    df_table = df_table.fillna('--------')
    return df_table

#меняем дату на сегодняшнюю
def update_date():
     #дата сегодня
    current_date = datetime.now()
    
    #дата в строку
    current_date_str = current_date.strftime('%Y.%m.%d')
    
    #меняем дату в ссылке
    updated_path = path.replace(re.search(r'start=\d{4}.\d{2}.\d{2}', path).group(), f'start={current_date_str}')
    
    #обновлённая ссылка
    return updated_path

#Главная функция
def job():
    # Обновление даты в пути
    updated_path = update_date()
    
    #Вызываем фукнции поменьше
    df_table = aspx_to_df(updated_path)
    df_result = generate_schedule(df_table)
    
    #Сохранение результата в png
    dfi.export(df_result.style.background_gradient(), 'schedule.png')
    
    # Отправка картинки в телегу
    send_telegram_message('schedule.png')

def send_telegram_message(image_path):
    #базированный токен и в какой чат отправляем
    token = '6791939546:AAEB_PMWhVZZ9VpSv1lbTd46eGf5X-gBQHc'
    chat_id = -1001431574416
    
    #Вытаскиваем картинку из папки
    with open(image_path, 'rb') as img:
        files = {'photo': ('schedule.png', img)}
        params = {'chat_id': chat_id}
        
        # Непосредственно закидываем картинку в телегу
        response = requests.post(f'https://api.telegram.org/bot{token}/sendPhoto', params=params, files=files)
    
    #Чистим картинку, чтобы не захламлять память
    if response.status_code == 200:
        os.remove(image_path)

In [4]:
job()